# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. Предоставлен набор данных с разметкой о токсичности правок.

Постройть модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Описание хода работы

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделайте выводы.

## Настройки рабочей тетради

In [1]:
# Импорт библиотек

import sys
#!{sys.executable} -m spacy download en

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import notebook
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from pymystem3 import Mystem
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

from sqlalchemy import create_engine 
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier

import spacy
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

import os

In [2]:
# Настройки библиотек

pd.options.mode.chained_assignment = None  # default='warn'
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [3]:
# Путь к директории с данными

path = 'datasets/'

## Подготовка

### Загрузка данных

In [4]:
data = pd.read_csv(os.path.join(path, 'toxic_comments.csv'))

display(data)
data.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data = data.drop('Unnamed: 0', axis=1)
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


<b>Вывод</b>

В исходных данных был столбец с индексами от предыдущей базы данных, некоторые значения в котором пропущены. Для обучаемых моделей в нем не было смысла, поэтому его удалили.

Полученные данные довольно громоздкие, больше 150000 строк.

### Подготовка выборок

- лемматизация
- создание признаков, а именно TF-IDF
- разделение на обучающую и тестовую выборки

In [6]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
for_lemm = []


for i in notebook.tqdm(data.index):
    doc = nlp(data.loc[i, 'text'])
    for_lemm.append(" ".join([token.lemma_ for token in doc]))
    
data['lemm_text'] = for_lemm
data


  0%|          | 0/159292 [00:00<?, ?it/s]

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation \n why the edit make under my user...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! he match this background colour I be s...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I be really not try to edit war . it..."
3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n More \n I can not make any real suggestio..."
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."
...,...,...,...
159287,""":::::And for the second time of asking, when ...",0,""" : : : : : and for the second time of asking ..."
159288,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself \n\n that be...
159289,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer \n\n Umm , there s no actual article f..."
159290,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


In [7]:
train, test = train_test_split(data, test_size=0.1)

In [8]:
features_train = train['lemm_text']
target_train = train['toxic']

feautures_test = test['lemm_text']
target_test = test['toxic']

In [9]:
stopwords = set(stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(train['lemm_text'])

#features_train = count_tf_idf.transform(train['lemm_text'])
features_test = count_tf_idf.transform(test['lemm_text'])

Так как исходные данные довольно большие, имеет смысл почистить память от ненужных нам данных.

In [10]:
del for_lemm
del data
#del train
del test

## Обучение

In [11]:
# Создание списков для хранения результатов моделей

final_score_list =[]

In [12]:
def print_result(grid, label=''):
    '''Функция вывода параметров и значений контрольных метрик'''
    
    print(label)
    print('Подобранные параметры:')
    print(grid.best_params_)
    print('F1-метрика на обучающей выборке:', grid.best_score_ )

    predictions = grid.predict(features_test)
    f1_test = f1_score(target_test, predictions)
    print('F1-метрика на тестовой выборке:', f1_test )

    roc = roc_auc_score(target_test, predictions)
    auc_list.append(roc)
    print('AUC-ROC:', roc)

### Модели классификации sklearn

In [13]:
linear_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("linear", LogisticRegression(max_iter=1000, C=10)),
    ]
)

In [14]:
scores = cross_val_score(linear_pipeline, features_train, target_train, cv=3, scoring='f1')
final_score = scores.mean()

print('Средняя оценка качества модели:', final_score)
final_score_list.append(final_score)

Средняя оценка качества модели: 0.7739598295414537


In [15]:
# Decision Tree Classifier

dtc_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("dtc", DecisionTreeClassifier(random_state=123)),
    ]
)

In [16]:
params = {
    'dtc__max_depth': [15, 50, 80],
    'dtc__class_weight':['balanced']
}

grid_dt = GridSearchCV(dtc_pipeline, params, n_jobs=-1, scoring = 'f1', verbose=3, cv=3)
grid_dt.fit(features_train, target_train)
#print_result(grid_dt, 'Decision Tree Classifier')

print('Средняя оценка качества модели дерева решений:', grid_dt.best_score_)
final_score_list.append(grid_dt.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Средняя оценка качества модели дерева решений: 0.6379121507610183


<b>Вывод</b>

Модель логистической регрессии уже выдаёт высокое значение метрики f1.

У модели дерева решений уже на старте довольно малое значение метрики f1. Маловероятно, что получится поднять это значение до нужного по условия путём перебора гиперпараметров. 

### Модели LGBMClassifier

In [17]:
# LGBMClassifier

lgb_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stopwords)),
        ("lgb", LGBMClassifier(random_state=123)),
    ]
)

params = {
    'lgb__max_depth': [80],
    'lgb__n_estimators': [15, 50, 80],
    'lgb__class_weight':['balanced']
}

grid_dt = GridSearchCV(lgb_pipeline, params, n_jobs=-1, scoring = 'f1', verbose=3, cv=3)
grid_dt.fit(features_train, target_train)
#print_result(grid_dt, 'Decision Tree Classifier')

print('Средняя оценка качества модели градиентного бустинга:', grid_dt.best_score_)
final_score_list.append(grid_dt.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Средняя оценка качества модели градиентного бустинга: 0.7329448887393131


Модели LGBMClassifier не хватает немного до нужного значения.

<b>Вывод</b>

При довольно больших параметрах и после длительного обучения модели, нужное значение метрики f1 достигнуто, однако стоит отметить тот факт, что значение контрольной метрики на тестовой выборке больше чем на обучающей, вызывает вопросы.

## Сравнение моделей

In [18]:
compare_models = pd.DataFrame()
compare_models['model'] = ['LogisticRegression', 'Decision Tree Classifier','LGBMClassifier']
compare_models['F1_train'] = final_score_list
display(compare_models)

,model,F1_train
0,LogisticRegression,0.773960
1,Decision Tree Classifier,0.637912
2,LGBMClassifier,0.732945


Очевидно, что наилучшая модель, судя по метрике f1, это модель логистической регрессии

In [19]:
#LogisticRegression

model = LogisticRegression(max_iter=1000, C=10)

features_train = count_tf_idf.transform(train['lemm_text'])
model.fit(features_train, target_train)
predictions = model.predict(features_test)

result = f1_score(target_test, predictions)
print('f1 лучше модели на тестовой выборке равно', result)

f1 лучше модели на тестовой выборке равно 0.7879432624113475


<b>Вывод</b>

Нужное значение на тестовой выборке получено. Наилучшая модель - LogisticRegression

## Выводы

1. Данные подготовлены, выделен TF-IDF как признак для обучения моделей.
2. Обучены различные модели из библиотек sklearn и lightgbm, подобраны нужные гиперпараметры для получения нужного значения контрольной метрики.
3. Заданное по условию значение метрики f1, >0.75, получено